In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier, AdaBoostClassifier, StackingClassifier
from sklearn.model_selection import cross_val_score, StratifiedKFold, train_test_split, cross_val_predict, GridSearchCV
from sklearn.metrics import accuracy_score, f1_score, make_scorer, roc_auc_score, confusion_matrix, classification_report
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder, RobustScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.decomposition import PCA
import random
from sklearn.neighbors import LocalOutlierFactor
from sklearn.feature_selection import RFECV

from sklearn.impute import SimpleImputer, KNNImputer

import tensorflow as tf
from tensorflow.keras import layers, models

from Functions_Classes import *

from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE, RandomOverSampler

from feature_engine.encoding import CountFrequencyEncoder

from xgboost import XGBClassifier

import matplotlib.pyplot as plt

pd.set_option('display.max_columns', None)
np.random.seed(0)

In [2]:
df = pd.read_excel("C:/Users/Cagan Deliktas/Desktop/ProjectDataMining2/DM2_DataCraft/data/training_data.xls")
X_test_compete = pd.read_excel("C:/Users/Cagan Deliktas/Desktop/ProjectDataMining2/DM2_DataCraft/data/test_data_no_target.xls")

df_whole = df.loc[:, df.columns != 'Perform']
#df = df.loc[:, df.columns != 'Group']


df_x = df.loc[:, df.columns != 'Class'].reset_index(drop=True)
df_y = df['Class'].reset_index(drop=True).reset_index(drop=True)

X_train = df_x.copy()
y_train =  df_y.copy()

#X_train, X_test, y_train, y_test = train_test_split(df_x, df_y, test_size=0.2,shuffle=True, stratify=df_y, random_state=0)

### Remove one year differences from the dataset

In [3]:
numeric_columns = X_train.loc[:, ~X_train.columns.isin(['Group'])].columns.to_list()
X_train[numeric_columns] = X_train.loc[:, numeric_columns].replace(
    {
        'NA': np.nan, 
        '': np.nan,
        ' ': np.nan
    }
).astype(float)

In [4]:
X_test_compete.head()

,Group,I1,I2,I3,I4,I5,I6,I7,I8,I9,I10,I11,I12,I13,I14,I15,I16,I17,I18,I19,I20,I21,I22,I23,I24,I25,I26,I27,I28,I29,I30,I31,I32,I33,I34,I35,I36,I37,I38,I39,I40,I41,I42,I43,I44,I45,I46,I47,I48,I49,I50,I51,I52,I53,I54,I55,I56,I57,I58,dI1,dI2,dI3,dI4,dI5,dI6,dI7,dI8,dI9,dI10,dI11,dI12,dI13,dI14,dI15,dI16,dI17,dI18,dI19,dI20,dI21,dI22,dI23,dI24,dI25,dI26,dI27,dI28,dI29,dI30,dI31,dI32,dI33,dI34,dI35,dI36,dI37,dI38,dI39,dI40,dI41,dI42,dI43,dI44,dI45,dI46,dI47,dI48,dI49,dI50,dI51,dI52,dI53,dI54,dI55,dI56,dI57,dI58
0,G3,0.039284,-0.034382,-0.040373,0.090023,-0.055953,0.049229,0.016236,-0.045865,0.692039,-0.027972,0.131225,-0.145072,-0.036565,-0.539252,0.157452,-0.239507,-0.117875,-0.044612,-0.121380,0.423125,-0.110934,-0.137485,-0.118075,-0.109066,-0.086454,0.000014,-0.471563,-0.327591,-0.130322,-0.066063,-0.457378,-0.962032,0.040413,0.040742,0.128125,-0.114141,0.010219,-0.108847,-0.059776,-0.374039,0.068467,0.034947,0.117171,-0.022869,0.099361,0.033746,-0.639667,-0.238583,-0.147187,-0.050275,0.092938,-0.460325,-0.118883,-1.098099,-0.903850,0.061494,-0.104035,-0.044155,-0.511172,-0.006080,-0.000742,-0.089311,0.099001,-0.010256,-0.027425,-0.001405,-0.132986,-0.011351,-0.103128,-0.285290,0.042478,-0.000351,-0.128966,-0.001738,0.003213,-0.000468,0.001363,-0.330708,0.008682,-0.002026,0.001667,0.001906,-0.018289,-0.041595,0.023208,0.000493,-0.013867,-0.010113,-0.128380,-0.050773,-0.098443,-0.097397,-0.041326,-0.368348,-0.303158,-0.000095,-0.012215,-0.151427,-0.119188,-0.127304,NaN,-0.031289,-0.115446,0.327623,-0.264556,0.004347,-0.377892,-0.410908,-0.417438,-0.220481,0.002526,0.124543,-0.219341,0.148324,0.020379,-0.004969
1,G9,-0.491713,-0.039484,-0.043612,-0.320312,0.065520,-0.216314,-0.366573,-0.052790,-0.370449,-0.031982,-0.394139,1.052776,-0.047382,-0.644620,-0.500026,-0.027195,-0.076228,-0.046266,-0.100503,-0.325795,-0.092976,-0.075757,-0.117135,-0.108077,0.078680,-0.157624,-0.489437,-0.332592,-0.149638,-0.078254,-0.826328,-1.129584,-0.671395,-0.663505,-0.719704,-0.577483,-0.696586,-0.162362,-0.083220,-0.841742,0.132480,0.120882,0.093526,-0.013213,0.701090,-0.120271,1.163667,-0.202620,0.387619,0.585477,0.951937,0.603286,-0.120165,-0.418795,-0.045985,0.932997,-0.065770,-0.052248,-0.630483,-0.010604,-0.005519,-0.107277,0.143069,-0.066005,-0.076414,-0.010868,-0.656338,-0.016636,-0.382414,0.747827,0.046481,-0.061869,-0.285885,0.210321,0.025196,-0.001145,-0.015368,-0.281431,-0.023203,-0.025029,-0.003940,-0.004397,-0.272864,-0.003262,-0.479521,-0.033243,0.024521,0.008407,0.039157,-0.067537,-0.129022,-0.127652,-0.048991,-0.269916,-0.187858,0.040417,0.009369,0.045425,-0.860801,-0.779291,-0.063318,0.121594,0.429946,-0.336888,1.165778,-0.047312,0.553799,0.660505,0.806000,0.781506,-0.012519,-0.014169,0.669341,0.015033,0.023411,0.007037
2,G7,-0.744979,-0.038271,-0.038631,-0.824021,0.117907,-1.434879,-0.707810,-0.073176,-1.698337,-0.035465,-0.911459,1.869291,0.023926,-0.157436,-1.141631,-0.320029,0.105402,-0.048712,0.249622,-0.551604,-0.048290,-0.227668,-0.091707,-0.082434,-0.794715,-0.076814,-0.717149,-0.519688,-0.020329,-0.039764,-0.027898,0.599326,-0.196814,-0.193965,0.404207,-0.210294,0.339902,-0.168858,-0.080228,-0.521322,0.042973,0.288377,0.803408,-0.002579,-0.676163,-0.714703,-0.960222,-0.808877,-0.071777,0.218954,-0.211625,-0.611026,0.065340,0.716997,1.087778,0.673804,-0.161341,1.076262,0.178010,0.006519,-0.001469,0.015004,0.118958,-0.828190,-0.875034,-0.031690,-0.866050,0.008078,-0.003916,2.544308,-0.269826,0.084440,-0.879775,-0.019950,0.002002,-0.000886,0.004123,0.294574,0.019364,0.001986,0.015836,0.017447,0.017963,-0.090231,-0.502316,0.000958,0.042708,0.013848,0.135136,-0.110357,-0.102783,-0.101692,-0.095080,0.403785,0.415998,0.001878,0.012861,0.141708,0.379565,2.016386,-0.724775,0.022578,0.631245,0.030817,-0.023111,-0.003808,0.707252,0.909110,0.738062,-0.024403,0.032079,0.095528,0.350733,-0.149682,0.005134,1.135389
3,G2,0.694507,-0.026038,-0.038985,1.892921,-0.100102,0.022436,-0.198048,-0.050572,0.208738,-0.014881,0.602613,-0.164873,-0.0

## Shape

In [5]:
X_train.shape

(8000, 118)

# Classification Models

In [6]:
nn = MLPClassifier(
    hidden_layer_sizes=(256, 128, 64),
    activation='relu',
    solver='adam',
    max_iter=100000, 
    random_state=0
)

voting_estimators = [
    ('RandomForest', RandomForestClassifier(random_state=0)),
    #('DecisionTree', DecisionTreeClassifier(random_state=0)),
    #('SVM', SVC(random_state=0, probability=True)),
    #('NaiveBayes', GaussianNB()),
    #('KNN', KNeighborsClassifier()),
    #('LogisticRegression', LogisticRegression(random_state=0, solver="saga", max_iter=1000)),
    #('AdaBoost', AdaBoostClassifier(random_state=0, algorithm='SAMME')),
    ('GradientBoost', GradientBoostingClassifier(random_state=0)),
    ('XGBoost', XGBClassifier(seed=0)),
    ('NeuralNetwork', nn) #hidden_layer_sizes=(20,20) for 2 hidden layers with 20 neurons each
]

vote_model = VotingClassifier(
    estimators=voting_estimators, 
    voting='hard'
)

stacking_estimators = [
    ('RandomForest', RandomForestClassifier(random_state=0)),
    #('DecisionTree', DecisionTreeClassifier(random_state=0)),
    #('SVM', SVC(random_state=0, probability=True)),
    #('NaiveBayes', GaussianNB()),
    #('KNN', KNeighborsClassifier()),
    #('AdaBoost', AdaBoostClassifier(random_state=0, algorithm='SAMME')),
    ('GradientBoost', GradientBoostingClassifier(random_state=0)),
    ('XGBoost', XGBClassifier(seed=0)),
    ('NeuralNetwork',nn)
]
meta_stack_classifier = LogisticRegression(random_state=0, solver="saga", max_iter=1000)

stacking_model = StackingClassifier(
    estimators=stacking_estimators, 
    final_estimator=meta_stack_classifier, 
    cv=5
)

estimators = [
    ('RandomForest', RandomForestClassifier(random_state=0)),
    #('DecisionTree', DecisionTreeClassifier(random_state=0)),
    #('SVM', SVC(random_state=0, probability=True)),
    #('NaiveBayes', GaussianNB()),
    #('KNN', KNeighborsClassifier()),
    #('LogisticRegression', LogisticRegression(random_state=0, solver="saga", max_iter=1000)),
    ('GradientBoost', GradientBoostingClassifier(random_state=0)),
    ('XGBoost', XGBClassifier(seed=0)),
    #('AdaBoost', AdaBoostClassifier(random_state=0, algorithm='SAMME'))
    #('Voting', vote_model),
    #('Stacking', stacking_model),
    #('NeuralNetwork', MLPClassifier(random_state=0, max_iter=1000)) # 2 hidden layers with 20 neurons each
]

In [7]:
group_list = X_test_compete.Group.unique()
group_list

array(['G3', 'G9', 'G7', 'G2', 'G4', 'G5', 'G1', 'G8', 'G6', 'G11', 'G10'],
      dtype=object)

In [8]:
preds_df = pd.DataFrame()
for group in group_list:
    print(group)
    df = df_whole.loc[df_whole.Group == group, :].copy()
    x_train = df.loc[:, ~df.columns.isin(['Class', 'Group'])].reset_index(drop=True).copy()
    y_train = df['Class'].reset_index(drop=True).reset_index(drop=True).copy()
    x_test = X_test_compete.loc[X_test_compete.Group == group, :].copy()
    x_test = x_test.drop('Group', axis=1)
    x_test_index = x_test.index

    if group in ['G10', 'G1']:
        x_train = x_train.drop(['I21', 'I48', 'I50', 'dI21', 'dI48', 'dI50'], axis=1)
        x_test = x_test.drop(['I21', 'I48', 'I50', 'dI21', 'dI48', 'dI50'], axis=1)
        
    #######################################
    X_trainP, imp = handle_missing_vals_simple(
        x_train
    )
    #######################################
    std_scale_cols = (
        X_trainP
        .loc[:, ~X_trainP.columns.str.contains('Group')]
        .columns
    )
    
    X_trainP, rbst_scaler = apply_robust_scaler(
        X_trainP, 
        std_scale_cols
    )
    #######################################
    X_trainP_df = pd.concat(
        [
            X_trainP.reset_index(drop=True), 
            pd.Series(y_train, name='Class').reset_index(drop=True)
        ], 
        axis=1
    )
    
    X_trainP_df = detect_outliers_with_lof(
        data=X_trainP_df
    )[0]
    
    X_trainP = (
        X_trainP_df
        .loc[:, X_trainP_df.columns != 'Class']
    )
    
    y_trainP = X_trainP_df['Class']
    #######################################
    X_trainP, y_trainP = apply_random_undersampling(X_trainP, y_trainP)
    #######################################
    #######################################
    X_test_competeP = pd.DataFrame(imp.transform(x_test),columns=x_test.columns)
    X_test_competeP = pd.DataFrame(rbst_scaler.transform(X_test_competeP), columns=X_test_competeP.columns)    
    #####################################################################################################################
    vote_model.fit(X_trainP, y_trainP)
    predictions = vote_model.predict(X_test_competeP)

    predictions_df = pd.DataFrame({'preds': predictions, 'index': x_test_index})

    preds_df = pd.concat([preds_df, predictions_df], axis=0)
    print("bitti")

G3
bitti
G9
bitti
G7
bitti
G2
bitti
G4
bitti
G5
bitti
G1
bitti
G8
bitti
G6
bitti
G11
bitti
G10
bitti


In [9]:
X_test_compete = X_test_compete.reset_index()
X_test_compete.head()

,index,Group,I1,I2,I3,I4,I5,I6,I7,I8,I9,I10,I11,I12,I13,I14,I15,I16,I17,I18,I19,I20,I21,I22,I23,I24,I25,I26,I27,I28,I29,I30,I31,I32,I33,I34,I35,I36,I37,I38,I39,I40,I41,I42,I43,I44,I45,I46,I47,I48,I49,I50,I51,I52,I53,I54,I55,I56,I57,I58,dI1,dI2,dI3,dI4,dI5,dI6,dI7,dI8,dI9,dI10,dI11,dI12,dI13,dI14,dI15,dI16,dI17,dI18,dI19,dI20,dI21,dI22,dI23,dI24,dI25,dI26,dI27,dI28,dI29,dI30,dI31,dI32,dI33,dI34,dI35,dI36,dI37,dI38,dI39,dI40,dI41,dI42,dI43,dI44,dI45,dI46,dI47,dI48,dI49,dI50,dI51,dI52,dI53,dI54,dI55,dI56,dI57,dI58
0,0,G3,0.039284,-0.034382,-0.040373,0.090023,-0.055953,0.049229,0.016236,-0.045865,0.692039,-0.027972,0.131225,-0.145072,-0.036565,-0.539252,0.157452,-0.239507,-0.117875,-0.044612,-0.121380,0.423125,-0.110934,-0.137485,-0.118075,-0.109066,-0.086454,0.000014,-0.471563,-0.327591,-0.130322,-0.066063,-0.457378,-0.962032,0.040413,0.040742,0.128125,-0.114141,0.010219,-0.108847,-0.059776,-0.374039,0.068467,0.034947,0.117171,-0.022869,0.099361,0.033746,-0.639667,-0.238583,-0.147187,-0.050275,0.092938,-0.460325,-0.118883,-1.098099,-0.903850,0.061494,-0.104035,-0.044155,-0.511172,-0.006080,-0.000742,-0.089311,0.099001,-0.010256,-0.027425,-0.001405,-0.132986,-0.011351,-0.103128,-0.285290,0.042478,-0.000351,-0.128966,-0.001738,0.003213,-0.000468,0.001363,-0.330708,0.008682,-0.002026,0.001667,0.001906,-0.018289,-0.041595,0.023208,0.000493,-0.013867,-0.010113,-0.128380,-0.050773,-0.098443,-0.097397,-0.041326,-0.368348,-0.303158,-0.000095,-0.012215,-0.151427,-0.119188,-0.127304,NaN,-0.031289,-0.115446,0.327623,-0.264556,0.004347,-0.377892,-0.410908,-0.417438,-0.220481,0.002526,0.124543,-0.219341,0.148324,0.020379,-0.004969
1,1,G9,-0.491713,-0.039484,-0.043612,-0.320312,0.065520,-0.216314,-0.366573,-0.052790,-0.370449,-0.031982,-0.394139,1.052776,-0.047382,-0.644620,-0.500026,-0.027195,-0.076228,-0.046266,-0.100503,-0.325795,-0.092976,-0.075757,-0.117135,-0.108077,0.078680,-0.157624,-0.489437,-0.332592,-0.149638,-0.078254,-0.826328,-1.129584,-0.671395,-0.663505,-0.719704,-0.577483,-0.696586,-0.162362,-0.083220,-0.841742,0.132480,0.120882,0.093526,-0.013213,0.701090,-0.120271,1.163667,-0.202620,0.387619,0.585477,0.951937,0.603286,-0.120165,-0.418795,-0.045985,0.932997,-0.065770,-0.052248,-0.630483,-0.010604,-0.005519,-0.107277,0.143069,-0.066005,-0.076414,-0.010868,-0.656338,-0.016636,-0.382414,0.747827,0.046481,-0.061869,-0.285885,0.210321,0.025196,-0.001145,-0.015368,-0.281431,-0.023203,-0.025029,-0.003940,-0.004397,-0.272864,-0.003262,-0.479521,-0.033243,0.024521,0.008407,0.039157,-0.067537,-0.129022,-0.127652,-0.048991,-0.269916,-0.187858,0.040417,0.009369,0.045425,-0.860801,-0.779291,-0.063318,0.121594,0.429946,-0.336888,1.165778,-0.047312,0.553799,0.660505,0.806000,0.781506,-0.012519,-0.014169,0.669341,0.015033,0.023411,0.007037
2,2,G7,-0.744979,-0.038271,-0.038631,-0.824021,0.117907,-1.434879,-0.707810,-0.073176,-1.698337,-0.035465,-0.911459,1.869291,0.023926,-0.157436,-1.141631,-0.320029,0.105402,-0.048712,0.249622,-0.551604,-0.048290,-0.227668,-0.091707,-0.082434,-0.794715,-0.076814,-0.717149,-0.519688,-0.020329,-0.039764,-0.027898,0.599326,-0.196814,-0.193965,0.404207,-0.210294,0.339902,-0.168858,-0.080228,-0.521322,0.042973,0.288377,0.803408,-0.002579,-0.676163,-0.714703,-0.960222,-0.808877,-0.071777,0.218954,-0.211625,-0.611026,0.065340,0.716997,1.087778,0.673804,-0.161341,1.076262,0.178010,0.006519,-0.001469,0.015004,0.118958,-0.828190,-0.875034,-0.031690,-0.866050,0.008078,-0.003916,2.544308,-0.269826,0.084440,-0.879775,-0.019950,0.002002,-0.000886,0.004123,0.294574,0.019364,0.001986,0.015836,0.017447,0.017963,-0.090231,-0.502316,0.000958,0.042708,0.013848,0.135136,-0.110357,-0.102783,-0.101692,-0.095080,0.403785,0.415998,0.001878,0.012861,0.141708,0.379565,2.016386,-0.724775,0.022578,0.631245,0.030817,-0.023111,-0.003808,0.707252,0.909110,0.738062,-0.024403,0.032079,0.095528,0.350733,-0.149682,0.005134,1.135389
3,3,G2,0.694507,-0.026038,-0.038985,1.892921,-0.100102,0.022436,-0.198048,-0.050572,0.208738,-0.014881,0.602613,

In [10]:
X_test_compete = pd.merge(X_test_compete, preds_df, how='inner', on='index')
preds_final_df = X_test_compete[['preds']]

In [11]:
file_path = "comb9.txt"
preds_final_df.to_csv(file_path, index=False, header=False)

In [12]:
preds_final_df

,preds
0,0
1,0
2,-1
3,-1
4,-1
...,...
1995,0
1996,0
1997,-1
1998,-1


In [13]:
X_test_compete

,index,Group,I1,I2,I3,I4,I5,I6,I7,I8,I9,I10,I11,I12,I13,I14,I15,I16,I17,I18,I19,I20,I21,I22,I23,I24,I25,I26,I27,I28,I29,I30,I31,I32,I33,I34,I35,I36,I37,I38,I39,I40,I41,I42,I43,I44,I45,I46,I47,I48,I49,I50,I51,I52,I53,I54,I55,I56,I57,I58,dI1,dI2,dI3,dI4,dI5,dI6,dI7,dI8,dI9,dI10,dI11,dI12,dI13,dI14,dI15,dI16,dI17,dI18,dI19,dI20,dI21,dI22,dI23,dI24,dI25,dI26,dI27,dI28,dI29,dI30,dI31,dI32,dI33,dI34,dI35,dI36,dI37,dI38,dI39,dI40,dI41,dI42,dI43,dI44,dI45,dI46,dI47,dI48,dI49,dI50,dI51,dI52,dI53,dI54,dI55,dI56,dI57,dI58,preds
0,0,G3,0.039284,-0.034382,-0.040373,0.090023,-0.055953,0.049229,0.016236,-0.045865,0.692039,-0.027972,0.131225,-0.145072,-0.036565,-0.539252,0.157452,-0.239507,-0.117875,-0.044612,-0.121380,0.423125,-0.110934,-0.137485,-0.118075,-0.109066,-0.086454,0.000014,-0.471563,-0.327591,-0.130322,-0.066063,-0.457378,-0.962032,0.040413,0.040742,0.128125,-0.114141,0.010219,-0.108847,-0.059776,-0.374039,0.068467,0.034947,0.117171,-0.022869,0.099361,0.033746,-0.639667,-0.238583,-0.147187,-0.050275,0.092938,-0.460325,-0.118883,-1.098099,-0.903850,0.061494,-0.104035,-0.044155,-0.511172,-0.006080,-0.000742,-0.089311,0.099001,-0.010256,-0.027425,-0.001405,-0.132986,-0.011351,-0.103128,-0.285290,0.042478,-0.000351,-0.128966,-0.001738,0.003213,-0.000468,0.001363,-0.330708,0.008682,-0.002026,0.001667,0.001906,-0.018289,-0.041595,0.023208,0.000493,-0.013867,-0.010113,-0.128380,-0.050773,-0.098443,-0.097397,-0.041326,-0.368348,-0.303158,-0.000095,-0.012215,-0.151427,-0.119188,-0.127304,NaN,-0.031289,-0.115446,0.327623,-0.264556,0.004347,-0.377892,-0.410908,-0.417438,-0.220481,0.002526,0.124543,-0.219341,0.148324,0.020379,-0.004969,0
1,1,G9,-0.491713,-0.039484,-0.043612,-0.320312,0.065520,-0.216314,-0.366573,-0.052790,-0.370449,-0.031982,-0.394139,1.052776,-0.047382,-0.644620,-0.500026,-0.027195,-0.076228,-0.046266,-0.100503,-0.325795,-0.092976,-0.075757,-0.117135,-0.108077,0.078680,-0.157624,-0.489437,-0.332592,-0.149638,-0.078254,-0.826328,-1.129584,-0.671395,-0.663505,-0.719704,-0.577483,-0.696586,-0.162362,-0.083220,-0.841742,0.132480,0.120882,0.093526,-0.013213,0.701090,-0.120271,1.163667,-0.202620,0.387619,0.585477,0.951937,0.603286,-0.120165,-0.418795,-0.045985,0.932997,-0.065770,-0.052248,-0.630483,-0.010604,-0.005519,-0.107277,0.143069,-0.066005,-0.076414,-0.010868,-0.656338,-0.016636,-0.382414,0.747827,0.046481,-0.061869,-0.285885,0.210321,0.025196,-0.001145,-0.015368,-0.281431,-0.023203,-0.025029,-0.003940,-0.004397,-0.272864,-0.003262,-0.479521,-0.033243,0.024521,0.008407,0.039157,-0.067537,-0.129022,-0.127652,-0.048991,-0.269916,-0.187858,0.040417,0.009369,0.045425,-0.860801,-0.779291,-0.063318,0.121594,0.429946,-0.336888,1.165778,-0.047312,0.553799,0.660505,0.806000,0.781506,-0.012519,-0.014169,0.669341,0.015033,0.023411,0.007037,0
2,2,G7,-0.744979,-0.038271,-0.038631,-0.824021,0.117907,-1.434879,-0.707810,-0.073176,-1.698337,-0.035465,-0.911459,1.869291,0.023926,-0.157436,-1.141631,-0.320029,0.105402,-0.048712,0.249622,-0.551604,-0.048290,-0.227668,-0.091707,-0.082434,-0.794715,-0.076814,-0.717149,-0.519688,-0.020329,-0.039764,-0.027898,0.599326,-0.196814,-0.193965,0.404207,-0.210294,0.339902,-0.168858,-0.080228,-0.521322,0.042973,0.288377,0.803408,-0.002579,-0.676163,-0.714703,-0.960222,-0.808877,-0.071777,0.218954,-0.211625,-0.611026,0.065340,0.716997,1.087778,0.673804,-0.161341,1.076262,0.178010,0.006519,-0.001469,0.015004,0.118958,-0.828190,-0.875034,-0.031690,-0.866050,0.008078,-0.003916,2.544308,-0.269826,0.084440,-0.879775,-0.019950,0.002002,-0.000886,0.004123,0.294574,0.019364,0.001986,0.015836,0.017447,0.017963,-0.090231,-0.502316,0.000958,0.042708,0.013848,0.135136,-0.110357,-0.102783,-0.101692,-0.095080,0.403785,0.415998,0.001878,0.012861,0.141708,0.379565,2.016386,-0.724775,0.022578,0.631245,0.030817,-0.023111,-0.003808,0.707252,0.909110,0.738062,-0.024403,0.032079,0.095528,0.350733,-0.149682,0.005134,1.135389,-1
3,3,G2,0.694507,-0.026038,-0.038985,1.892921,-0.100102,0.022436,-0.198048,-0.050572,0.208738,-0.014